# Análise de Componentes Principais 

##### EXPERIMENTO I:
    PCA PARA OS 5 CLASSIFICADORES ONDE O TREINAMENTO É FEITO COM O DATASET ORIGINAL E O TESTE COM O DATASET ORIGINAL

##### EXEPRIMENTO II:
    PCA PARA OS 5 CLASSIFICADORES ONDE O TREINAMENTO É FEITO COM O DATASET COMPLETO E O TESTE COM O DATASET ORIGINAL

##### EXEPRIMENTO III:
    PCA PARA OS 5 CLASSIFICADORES ONDE O TREINAMENTO É FEITO COM O DATASET ORIGINAL E O TESTE COM O DATASET COMPLETO

##### EXEPRIMENTO IV:
    PCA PARA OS 5 CLASSIFICADORES ONDE O TREINAMENTO É FEITO COM O DATASET COMPLETO E O TESTE COM O DATASET COMPLETO

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import shutil
import os
from joblib import Parallel, delayed
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split as tts
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score

In [ ]:
#DEFININDO ALGUNS PARÂMETROS DO GRÁFICO
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
params = {
    'figure.figsize': [11, 5.5], 
    'axes.labelsize': 18,
    'axes.titlesize':20, 
    'font.size': 18,
    'legend.fontsize': 14, 
    'xtick.labelsize': 12, 
    'ytick.labelsize': 12,
    'axes.axisbelow': True
}
plt.rcParams.update(params)

## Parâmetros Iniciais

In [ ]:
caminhoCSVTreinamentoCompleto = "/home/dimi/Programming/MachineLearningUFABC/Projeto_Final/datasets/500ms_60_features/dataset_treinamento_completo.csv"
caminhoCSVTesteCompleto       = "/home/dimi/Programming/MachineLearningUFABC/Projeto_Final/datasets/500ms_60_features/dataset_teste_completo.csv"

arrayClassificadores = ["KNN", "SVM", "MLP", "RF", "LDA"]
caminhoOndeSalvar = "resultados/"

resultadosRapidos = False # REDUZ O DATASET A 0.30 CASO TRUE (COLOCAR TRUE APENAS PARA TESTES RAPIDOS)

## Deletando qualquer resultado anterior

In [ ]:
if os.path.isdir(caminhoOndeSalvar+"PCA/") == True:
    shutil.rmtree(caminhoOndeSalvar+"PCA/")
    
os.makedirs(caminhoOndeSalvar+"PCA/")

## Definição de funções

In [ ]:
def ajustarDataset(dataframe, efeitosDesejados):
    
    if not efeitosDesejados in ["original", "original+ruido", "original+pitch", "original+tempo", "original+velocidade", "completo"]:
        print("Escolha dentre os seguintes efeitos: original, original+ruido, original+pitch, original+tempo, original+velocidade ou completo!")
        return None, None, None
    
    else:
        if efeitosDesejados == "original":
            dataframeAjustado = dataframe.loc[(dataframe['ruido']==0)&(dataframe['pitch']==0)&(dataframe['tempo']==0)&(dataframe['velocidade']==0)]
        elif efeitosDesejados == "original+ruido":
            dataframeAjustado = pd.concat([dataframe.loc[(dataframe['ruido']==1)&(dataframe['pitch']==0)&(dataframe['tempo']==0)&(dataframe['velocidade']==0)], dataframe.loc[(dataframe['ruido']==0)&(dataframe['pitch']==0)&(dataframe['tempo']==0)&(dataframe['velocidade']==0)]], ignore_index=True)
        elif efeitosDesejados == "original+pitch":
            dataframeAjustado = pd.concat([dataframe.loc[(dataframe['ruido']==0)&(dataframe['pitch']==1)&(dataframe['tempo']==0)&(dataframe['velocidade']==0)], dataframe.loc[(dataframe['ruido']==0)&(dataframe['pitch']==0)&(dataframe['tempo']==0)&(dataframe['velocidade']==0)]], ignore_index=True)
        elif efeitosDesejados == "original+tempo":
            dataframeAjustado = pd.concat([dataframe.loc[(dataframe['ruido']==0)&(dataframe['pitch']==0)&(dataframe['tempo']==1)&(dataframe['velocidade']==0)], dataframe.loc[(dataframe['ruido']==0)&(dataframe['pitch']==0)&(dataframe['tempo']==0)&(dataframe['velocidade']==0)]], ignore_index=True)
        elif efeitosDesejados == "original+velocidade":
            dataframeAjustado = pd.concat([dataframe.loc[(dataframe['ruido']==0)&(dataframe['pitch']==0)&(dataframe['tempo']==0)&(dataframe['velocidade']==1)], dataframe.loc[(dataframe['ruido']==0)&(dataframe['pitch']==0)&(dataframe['tempo']==0)&(dataframe['velocidade']==0)]], ignore_index=True)
        else:
            dataframeAjustado = dataframe
            
        # SEPARO O QUE E O QUE
        data   = dataframeAjustado.iloc[:, 5:-1].to_numpy()
        target = dataframeAjustado.iloc[:, -1].to_numpy()
        
        del dataframeAjustado
        return data, target

In [ ]:
def aplicarPCA(xTrain, xTest, qtdDimensoes):
    
    objPCA = PCA(n_components=qtdDimensoes).fit(xTrain)
    xTrainPCA = objPCA.transform(xTrain)
    xTestPCA  = objPCA.transform(xTest)
    
    del objPCA
    return xTrainPCA, xTestPCA

In [ ]:
def treinarEClassificar(xTrain, xTest, yTrain, yTest, classificador):
    
    if classificador == "KNN":
        objClassificador = KNeighborsClassifier(n_neighbors=1)
    elif classificador == "LDA":
        objClassificador = LinearDiscriminantAnalysis()
    elif classificador == "RF":
        objClassificador = RandomForestClassifier(n_estimators=10)
    elif classificador == "SVM":
        objClassificador = SVC(gamma="scale")
    else:
        objClassificador = MLPClassifier(hidden_layer_sizes=())
    
    objClassificador.fit(xTrain, yTrain)
    yPred = objClassificador.predict(xTest)
    
    del objClassificador
    return accuracy_score(yTest, yPred)

In [ ]:
def salvarArrayAcuraciaCadaDimensao(arrayAcuraciaCadaDimensao, acuraciaReferencial, experimento, classificador, caminhoOndeSalvar):
    
    # ABRINDO O ARQUIVO TXT
    objFile = open(caminhoOndeSalvar+"PCA/resultados_PCA.txt", "a")

    # ESCREVENDO O CABECALHO
    objFile.write("EXPERIMENTO " + str(experimento) + " - " + classificador + ":\n")
    
    # ESCREVENDO A ACURACIA SEM PCA (REFERENCIAL)
    objFile.write("Acurácia referencial (sem PCA): " + str(acuraciaReferencial) + "\n")

    # ESCREVENDO O RESULTADO DE CADA DIMENSAO
    objFile.write("Acurácia com PCA para cada dimensionalidade: ")
    for acuraciaAtual in arrayAcuraciaCadaDimensao:
        objFile.write(str(acuraciaAtual) + ", ")
    objFile.write("\n\n")

    # FECHANDO
    objFile.close()    

In [ ]:
def printarGrafico(arrayAcuraciaCadaDimensao, acuraciaReferencial, experimento, classificador, caminhoOndeSalvar):
    
    plt.plot(np.arange(1,len(arrayAcuraciaCadaDimensao)+1), np.full(len(arrayAcuraciaCadaDimensao), acuraciaReferencial), "--", color="black", label="Referencial (sem PCA)")
    plt.plot(np.arange(1,len(arrayAcuraciaCadaDimensao)+1), arrayAcuraciaCadaDimensao, label="PCA")
    plt.title("EXPERIMENTO " + str(experimento) + " - " + classificador)
    plt.xlabel("Qtd de Dimensões")
    plt.ylabel("Acurácia")
    plt.grid(alpha=0.5)
    plt.legend()
    plt.savefig(caminhoOndeSalvar+"PCA/experimento"+str(experimento)+"_"+classificador+".pdf", format='pdf', bbox_inches='tight')
    plt.show()

In [ ]:
def aplicarPCA(classificador, qtdDimensoes, xTrain, xTest, yTrain, yTest):
    # APLICO O PCA
    objPCA    = PCA(n_components=qtdDimensoes).fit(xTrain)
    xTrainPCA = objPCA.transform(xTrain)
    xTestPCA  = objPCA.transform(xTest)

    # CALCULO A ACURACIA COM A REDUCAO
    return treinarEClassificar(xTrainPCA, xTestPCA, yTrain, yTest, classificador)

In [ ]:
def testarUmClassificador(experimento, classificador, xTrain, xTest, yTrain, yTest, caminhoOndeSalvar):
    print("Iniciando os testes com o classificador", classificador)
    
    # OBTENDO A ACURACIA REFERENCIAL (SEM PCA)
    acuraciaReferencial = treinarEClassificar(xTrain, xTest, yTrain, yTest, classificador)
    # ITERANDO AS DIMENSOES DO PCA
    arrayAcuraciaCadaDimensao = Parallel(n_jobs=-1, verbose=10)(delayed(aplicarPCA)(classificador, qtdDimensoesAtual, xTrain, xTest, yTrain, yTest) for qtdDimensoesAtual in range(1, len(xTest[0])))
    
    # NESSE PONTO EU JA VOU TER O RESULTADO FINAL PARA O CLASSIFICADOR ATUAL NO EXPERIMENTO ATUAL, VOU SALVAR AS INFORMACOES
    salvarArrayAcuraciaCadaDimensao(arrayAcuraciaCadaDimensao, acuraciaReferencial, experimento, classificador, caminhoOndeSalvar)
    printarGrafico(arrayAcuraciaCadaDimensao, acuraciaReferencial, experimento, classificador, caminhoOndeSalvar)
    
    print("Todos os testes com o classificador", classificador, "foram finalizados")

## Experimento I

EXPERIMENTO I:
PCA PARA OS 5 CLASSIFICADORES ONDE O TREINAMENTO É FEITO COM O DATASET ORIGINAL E O TESTE COM O DATASET ORIGINAL

##### Parâmetros

In [ ]:
experimentoAtual  = 1
efeitoTreinamento = "original"
efeitoTeste       = "original"

##### Rodando

In [ ]:
dfTreinamento  = pd.read_csv(caminhoCSVTreinamentoCompleto)
dfTeste        = pd.read_csv(caminhoCSVTesteCompleto)
xTrain, yTrain = ajustarDataset(dfTreinamento, efeitoTreinamento)
xTest, yTest   = ajustarDataset(dfTeste, efeitoTeste)
del dfTreinamento
del dfTeste

if resultadosRapidos == True:
    xTrain, yTrain = shuffle(xTrain, yTrain) # TESTES RAPIDOS
    xTest, yTest   = shuffle(xTest, yTest) # TESTES RAPIDOS
    xTrain = xTrain[:int(0.3*len(yTest))] # TESTES RAPIDOS
    yTrain = yTrain[:int(0.3*len(yTest))] # TESTES RAPIDOS
    xTest  = xTest[:int(0.3*len(yTest))] # TESTES RAPIDOS
    yTest  = yTest[:int(0.3*len(yTest))] # TESTES RAPIDOS

# RODANDO A ANALISE PARA CADA CLASSIFICADOR
Parallel(n_jobs=-1, verbose=10)(delayed(testarUmClassificador)(experimentoAtual, classificadorAtual, xTrain, xTest, yTrain, yTest, caminhoOndeSalvar) for classificadorAtual in arrayClassificadores)
    
# LIMPO A MEMORIA
del xTrain
del xTest
del yTrain
del yTest

## Experimento II

EXEPRIMENTO II:
PCA PARA OS 5 CLASSIFICADORES ONDE O TREINAMENTO É FEITO COM O DATASET COMPLETO E O TESTE COM O DATASET ORIGINAL

##### Parâmetros

In [ ]:
experimentoAtual  = 2
efeitoTreinamento = "completo"
efeitoTeste       = "original"

##### Rodando

In [ ]:
dfTreinamento  = pd.read_csv(caminhoCSVTreinamentoCompleto)
dfTeste        = pd.read_csv(caminhoCSVTesteCompleto)
xTrain, yTrain = ajustarDataset(dfTreinamento, efeitoTreinamento)
xTest, yTest   = ajustarDataset(dfTeste, efeitoTeste)
del dfTreinamento
del dfTeste

if resultadosRapidos == True:
    xTrain, yTrain = shuffle(xTrain, yTrain) # TESTES RAPIDOS
    xTest, yTest   = shuffle(xTest, yTest) # TESTES RAPIDOS
    xTrain = xTrain[:int(0.3*len(yTest))] # TESTES RAPIDOS
    yTrain = yTrain[:int(0.3*len(yTest))] # TESTES RAPIDOS
    xTest  = xTest[:int(0.3*len(yTest))] # TESTES RAPIDOS
    yTest  = yTest[:int(0.3*len(yTest))] # TESTES RAPIDOS

# RODANDO A ANALISE PARA CADA CLASSIFICADOR
Parallel(n_jobs=-1, verbose=10)(delayed(testarUmClassificador)(experimentoAtual, classificadorAtual, xTrain, xTest, yTrain, yTest, caminhoOndeSalvar) for classificadorAtual in arrayClassificadores)
    
# LIMPO A MEMORIA
del xTrain
del xTest
del yTrain
del yTest

## Experimento III

EXEPRIMENTO III:
PCA PARA OS 5 CLASSIFICADORES ONDE O TREINAMENTO É FEITO COM O DATASET ORIGINAL E O TESTE COM O DATASET COMPLETO

##### Parâmetros

In [ ]:
experimentoAtual  = 3
efeitoTreinamento = "original"
efeitoTeste       = "completo"

##### Rodando

In [ ]:
dfTreinamento  = pd.read_csv(caminhoCSVTreinamentoCompleto)
dfTeste        = pd.read_csv(caminhoCSVTesteCompleto)
xTrain, yTrain = ajustarDataset(dfTreinamento, efeitoTreinamento)
xTest, yTest   = ajustarDataset(dfTeste, efeitoTeste)
del dfTreinamento
del dfTeste

if resultadosRapidos == True:
    xTrain, yTrain = shuffle(xTrain, yTrain) # TESTES RAPIDOS
    xTest, yTest   = shuffle(xTest, yTest) # TESTES RAPIDOS
    xTrain = xTrain[:int(0.3*len(yTest))] # TESTES RAPIDOS
    yTrain = yTrain[:int(0.3*len(yTest))] # TESTES RAPIDOS
    xTest  = xTest[:int(0.3*len(yTest))] # TESTES RAPIDOS
    yTest  = yTest[:int(0.3*len(yTest))] # TESTES RAPIDOS

# RODANDO A ANALISE PARA CADA CLASSIFICADOR
Parallel(n_jobs=-1, verbose=10)(delayed(testarUmClassificador)(experimentoAtual, classificadorAtual, xTrain, xTest, yTrain, yTest, caminhoOndeSalvar) for classificadorAtual in arrayClassificadores)
    
# LIMPO A MEMORIA
del xTrain
del xTest
del yTrain
del yTest

## Experimento IV

EXEPRIMENTO IV:
PCA PARA OS 5 CLASSIFICADORES ONDE O TREINAMENTO É FEITO COM O DATASET COMPLETO E O TESTE COM O DATASET COMPLETO

##### Parâmetros

In [ ]:
experimentoAtual  = 4
efeitoTreinamento = "completo"
efeitoTeste       = "completo"

##### Rodando

In [ ]:
dfTreinamento  = pd.read_csv(caminhoCSVTreinamentoCompleto)
dfTeste        = pd.read_csv(caminhoCSVTesteCompleto)
xTrain, yTrain = ajustarDataset(dfTreinamento, efeitoTreinamento)
xTest, yTest   = ajustarDataset(dfTeste, efeitoTeste)
del dfTreinamento
del dfTeste

if resultadosRapidos == True:
    xTrain, yTrain = shuffle(xTrain, yTrain) # TESTES RAPIDOS
    xTest, yTest   = shuffle(xTest, yTest) # TESTES RAPIDOS
    xTrain = xTrain[:int(0.3*len(yTest))] # TESTES RAPIDOS
    yTrain = yTrain[:int(0.3*len(yTest))] # TESTES RAPIDOS
    xTest  = xTest[:int(0.3*len(yTest))] # TESTES RAPIDOS
    yTest  = yTest[:int(0.3*len(yTest))] # TESTES RAPIDOS

# RODANDO A ANALISE PARA CADA CLASSIFICADOR
Parallel(n_jobs=-1, verbose=10)(delayed(testarUmClassificador)(experimentoAtual, classificadorAtual, xTrain, xTest, yTrain, yTest, caminhoOndeSalvar) for classificadorAtual in arrayClassificadores)
    
# LIMPO A MEMORIA
del xTrain
del xTest
del yTrain
del yTest